In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import time
import os

path="../data/"

os.getcwd()

'/home/cseadmin/undergrad1/dz/GISCUP2021/model'

In [2]:
start = time.time()

train = pd.read_pickle(path+'train/train.pkl')
test = pd.read_pickle(path+'test/test.pkl')

end = time.time()
print("Read data time = {:.2f}s".format(end-start))

print(train.info())
print()
print(test.info())

Read data time = 0.57s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8651005 entries, 0 to 8651004
Data columns (total 68 columns):
 #   Column             Dtype         
---  ------             -----         
 0   slice_id           int16         
 1   order_id           int32         
 2   ata                int16         
 3   distance           float32       
 4   simple_eta         int16         
 5   driver_id          int32         
 6   speed              float16       
 7   speed_avg          float16       
 8   speed_std          float16       
 9   speed_min          float16       
 10  speed_max          float16       
 11  volume             int16         
 12  old_driver         int8          
 13  link_no            int16         
 14  link_time_sum      float16       
 15  link_time_avg      float16       
 16  link_time_std      float16       
 17  link_time_max      float16       
 18  link_time_min      float16       
 19  next_no_1.0        float16       
 20  n

In [3]:
num_cols = [x for x in train.columns if x not in ['order_id', 'ata','date', 'driver_id',
                                                  'arrival_0','arrival_1','arrival_2','arrival_3','arrival_4']]
cat_cols = []
features = num_cols + cat_cols

train_x = train[features]
train_y = np.log(train['ata'])

test_x = test[features]

print(len(features), features, sep="\n")

64
['slice_id', 'distance', 'simple_eta', 'speed', 'speed_avg', 'speed_std', 'speed_min', 'speed_max', 'volume', 'old_driver', 'link_no', 'link_time_sum', 'link_time_avg', 'link_time_std', 'link_time_max', 'link_time_min', 'next_no_1.0', 'next_no_2.0', 'next_no_3.0', 'next_no_4.0', 'next_time_sum_1.0', 'next_time_sum_2.0', 'next_time_sum_3.0', 'next_time_sum_4.0', 'next_time_avg_1.0', 'next_time_avg_2.0', 'next_time_avg_3.0', 'next_time_avg_4.0', 'next_time_std_1.0', 'next_time_std_2.0', 'next_time_std_3.0', 'next_time_std_4.0', 'next_time_max_1.0', 'next_time_max_2.0', 'next_time_max_3.0', 'next_time_max_4.0', 'next_time_min_1.0', 'next_time_min_2.0', 'next_time_min_3.0', 'next_time_min_4.0', 'cross_no', 'cross_sum', 'cross_avg', 'cross_std', 'cross_max', 'cross_min', 'link_status_no_0', 'link_status_no_1', 'link_status_no_2', 'link_status_no_3', 'link_status_no_4', 'day_bias', 'weekday', 'link_length', 'log_distance', 'log_simple_eta', 'next_ratio_1', 'next_time_ratio_1', 'next_ratio

In [4]:
days = train['date'].unique()
tr_days, val_days = train_test_split(days, test_size=0.2, random_state=416)

tr_x = train_x[train['date'].isin(tr_days)]
tr_y = train_y[train['date'].isin(tr_days)]
val_x = train_x[train['date'].isin(val_days)]
val_y = train_y[train['date'].isin(val_days)]

# tr_x, val_x, tr_y, val_y = train_test_split(train_x,train_y,test_size=0.2,random_state=416)

train_set = lgb.Dataset(tr_x, tr_y, categorical_feature=cat_cols)
val_set = lgb.Dataset(val_x, val_y, categorical_feature=cat_cols)

In [8]:
lgb_paras = {
    'objective': 'regression',
    'learning_rate': 0.1,
    # 'num_leaves': 40,
    'max_depth': -1,

    'reg_alpha': 0.01,
    'reg_lambda': 0.02,

    'random_state': 416,

    'colsample_bytree': 1, # feature_fraction
    'subsample': 0.8, # bagging_fraction
    'subsample_freq': 3, # bagging_freq

    'n_estimators': 100,
    # 15712

    'n_jobs': 4,

    'silent': False
}

model=lgb.LGBMRegressor()
model.set_params(**lgb_paras)

LGBMRegressor(colsample_bytree=1, n_jobs=4, objective='regression',
              random_state=416, reg_alpha=0.01, reg_lambda=0.02, silent=False,
              subsample=0.8, subsample_freq=3)

In [9]:
# model.fit(tr_x, tr_y, eval_set=list(zip(val_x.values.tolist(), val_y.values.reshape(-1, 1).tolist())), eval_metric='rmse', early_stopping_rounds=100, verbose=10, categorical_feature=cat_cols)

# model.fit(train_x, train_y)

In [10]:
params_test1 = {
    'max_depth': range(3, 8, 2),
    'num_leaves':range(30, 90, 20)
}
gsearch1 = GridSearchCV(estimator=model, param_grid=params_test1, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=4)

start = time.time()

gsearch1.fit(tr_x, tr_y)

end = time.time()
print("Grid search time = {:.2f}s".format(end-start))

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits
/home/cseadmin/anaconda3/envs/dz/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 